# markdown.obisidian.personal.index_notes
> Functions for managing index notes one's Obsidian.md math vault.

In a Obsidian math vault, it is convenient to keep index notes, which list links to other index notes or standard information notes. 

The methods in this module (semi)automatically 
- create (standard information) notes in appropriate folders,
- set up the notes,
- add links of the notes to appropriate index notes
- add to both the index note and the standard information note a footnote indicating where the content of the information note originates from in the original text, e.g.  
`[[foag_transition_function_for_a_vector_bundle_over_a_manifold]], 13.1.1`, `[[bredon_tensor_product_of_graded_groups]], Page 315`, 
`[[eilenberg_zilber_theorem|Eilenberg Zilber theorem]], Theorem 1.3, Corollary 1.4`

In [40]:
#| default_exp markdown.obsidian.personal.index_notes

In [41]:
#| export
import glob
from natsort import natsorted
import os
from os import PathLike
from pathlib import Path
import re
from trouver.markdown.markdown.file import (
    MarkdownFile, MarkdownLineEnum
)
from trouver.markdown.markdown.heading import heading_title
from trouver.markdown.obsidian.links import (
    find_links_in_markdown_text, ObsidianLink, links_from_text
)
from trouver.markdown.obsidian.vault import (
    VaultNote, note_name_unique, note_path_by_name
)
from typing import Union

In [42]:
from unittest import mock

## Automatically filling in some notes

- I need to identify the names of subsections/subchapters inside a chapter, and identify the correspondence of subdirectories in the directory with headings in the index note.
- I need to use regex to find Theorems/Corollaries/Propositions/Lemmas/Definitions/Remarks/Examples (I'll refere to these as Numberings) in LaTeX code that has been OCR'd
- For each subsection/subchapter, I need to collect these Numberings, create an information note for each one, and add the number/page number.

In [43]:
#| export
def subsections_listed_in_index_note(
        index_note: Union[VaultNote, str], # The index note
        vault: PathLike
        ) -> dict[Union[int, str], [dict, str]]: # The keys are 1. line numbers and 2. `'title'`. The values are dict and str (the blank str if root node), respectively.
    """
    Return subsections/subchapters as listed in the index note

    **Parameters**
    - `index_note` - VaultNote or str
    - `vault` - Pathlike

    **Returns**
    - dict[Union[int, str], [dict, str]]
        
    """
    vault = Path(vault)
    if isinstance(index_note, str):
        index_note = VaultNote(vault, name=index_note)
    mf_file = MarkdownFile.from_vault_note(index_note)
    return mf_file.get_headings_tree()

In [44]:
#| export
def subsection_folders(
        index_note: Union[VaultNote, str],
        vault: PathLike,
        data_type: str) -> list[str]:
    """Returns subdirectories corresponding to subsections/subchapters, i.e.
    the folders in the same directory as the index note.
    
    **Parameters**
    - `index_note` - VaultNote or str
        - The (unique) name of the index note in the vault.
    - `vault` - PathLike
    - `data_type` - str
        - `'absolute_path'`, `'relative_path'`, or `'name'`
    
    **Returns**
    - list[str]
        - List of immediate subdirectories in the directory containing the
        index note.
    """
    vault = Path(vault)
    if isinstance(index_note, str):
        index_note = VaultNote(vault, name=index_note)
    parent_directory = (vault / index_note.rel_path).parent
    print(str(parent_directory))
    glob_result = list(glob.glob(str(parent_directory) + '/**/'))
    if data_type == 'absolute_path':
        return glob_result
    elif data_type == 'relative_path':
        return [str(Path(dir).relative_to(vault)) for dir in glob_result]
    elif data_type == 'name':
        return [Path(dir).name for dir in glob_result]
    

In [45]:
# with (mock.patch("trouver.markdown.obsidian.vault"))
# with (mock.patch("trouver.markdown.obsidian.vault"))
# with (mock.patch("trouver.markdown.markdown.file._io.TextIOWrapeer.read") as mock_read):
#     mock_read.return_value = text

with mock.patch("trouver.markdown.markdown.file.open", mock.mock_open(read_data='hi')):
    fake_vn = VaultNote(rel_path='fake_note.md', vault='')
    print(subsections_listed_in_index_note(fake_vn, vault=''))

# with mock.patch("__main__.open", mock.mock_open(read_data='hi')):
#     with open('styles.css') as f:
#         print(f.read())

# with mock.patch("trouver.markdown.markdown.file.VaultNote", mock.mock)

# subsections_listed_in_index_note('_index_VI_products_and_duality', MATH_VAULT_LOCATION)
# subsection_folders('_index_VI_products_and_duality', MATH_VAULT_LOCATION, data_type='name')

{'title': ''}


In [46]:
#| hide

# mock patch a somewhat actual example

text = r"""# 1. The Cross Product and the Kiinneth Theorem
- [ ] [[bredon_ 1.1_page_316]],  1.1, Page 316
- #_meta/TODO this is just a todo tag, It's not an actual heading!
# 2. A Sign Convention
- [[bredon_ 1.6_page_320_6]]
# 3. The Cohomology Cross Product
- [ ] [[bredon_ 3.1_page_323]],  3.1, Page 323
# 4. The Cup Product
- [ ] [[bredon_ 4.1_page_326]],  4.1, Page 326
# 5. The Cap Product
- [ ] [[bredon_ 5.1_page_335]],  5.1, Page 335
# 6. Classical Outlook on Duality
# 7. The Orientation Bundle
- [ ] [[bredon_ 7.1_page_341]],  7.1, Page 341
# 8. Duality Theorems
- [ ] [[bredon_ 8.1_page_349]],  8.1, Page 349
# 9. Duality on Compact Manifolds with Boundary
- [ ] [[bredon_ 9.1_page_356]],  9.1, Page 356
# 10. Applications of Duality
- [ ] [[bredon_ 10.1_page_359]],  10.1, Page 359
# 11. Intersection Theory
- [ ] [[bredon_ 11.1_page_368]],  11.1, Page 368
# 12. The Euler Class, Lefschetz Numbers, and Vector Fields
- [ ] [[bredon_ 12.1_page_378]],  12.1, Page 378
# 13. The Gysin Sequence
- [ ] [[bredon_ 13.1_page_390]],  13.1, Page 390
# 14. Lefschetz Coincidence Theory
- [ ] [[bredon_ 14.1_page_394]],  14.1, Page 394
# 15. Steenrod Operations
- [ ] [[bredon_ 15.1_page_405]],  15.1, Page 405
# 16. Construction of the Steenrod Squares
- [ ] [[bredon_ 16.1_page_414]],  16.1, Page 414
# 17. Stiefel-Whitney Classes
- [ ] [[bredon_ 17.1_page_421]],  17.1, Page 421
# 18. Plumbing ¥"""

with mock.patch("trouver.markdown.markdown.file.open", mock.mock_open(read_data=text)):
    fake_vn = VaultNote(rel_path='fake_note.md', vault='')
    subsections_in_text = subsections_listed_in_index_note(fake_vn, vault='')
    print(subsections_in_text)
    print(subsections_in_text[0])
    print(subsections_in_text['title'])


{'title': '', 0: {'title': '# 1. The Cross Product and the Kiinneth Theorem'}, 3: {'title': '# 2. A Sign Convention'}, 5: {'title': '# 3. The Cohomology Cross Product'}, 7: {'title': '# 4. The Cup Product'}, 9: {'title': '# 5. The Cap Product'}, 11: {'title': '# 6. Classical Outlook on Duality'}, 12: {'title': '# 7. The Orientation Bundle'}, 14: {'title': '# 8. Duality Theorems'}, 16: {'title': '# 9. Duality on Compact Manifolds with Boundary'}, 18: {'title': '# 10. Applications of Duality'}, 20: {'title': '# 11. Intersection Theory'}, 22: {'title': '# 12. The Euler Class, Lefschetz Numbers, and Vector Fields'}, 24: {'title': '# 13. The Gysin Sequence'}, 26: {'title': '# 14. Lefschetz Coincidence Theory'}, 28: {'title': '# 15. Steenrod Operations'}, 30: {'title': '# 16. Construction of the Steenrod Squares'}, 32: {'title': '# 17. Stiefel-Whitney Classes'}, 34: {'title': '# 18. Plumbing ¥'}}
{'title': '# 1. The Cross Product and the Kiinneth Theorem'}

